<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/cart_pole_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [2]:
!pip install pyglet==1.3.2

     |████████████████████████████████| 1.0MB 5.0MB/s 
  Found existing installation: pyglet 1.4.4
    Uninstalling pyglet-1.4.4:
      Successfully uninstalled pyglet-1.4.4


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
#gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
# You can just ignore the warning 'xdpyinfo was not found' for now.

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
# Utility functions to enable video recording of gym environment and displaying
# it. To enable video, just do:
#   "env = wrap_env(env)"

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
# Where ENV_NAME is the environment that are using from Gym, eg 'CartPole-v0'
ENV_NAME = 'CartPole-v0'
env = wrap_env(gym.make(ENV_NAME)) #wrapping the env to render as a video

In [7]:
# Check out the action and observation spaces.
#
# According to the GitHub
# (https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py),
# the discrete action space is defined as follows:
#
#     0 - Push the cart to the left
#     1 - Push the cart to the right
#
# and the continuous observation (state) space is defined as follows:
#
#     Num	Observation               Min             Max
#       0	Cart Position             -4.8            4.8
#       1	Cart Velocity             -Inf            Inf
#       2	Pole Angle                 -24 deg        24 deg
#       3	Pole Velocity At Tip      -Inf            Inf
#
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [15]:
# The video that is generated ends very quickly. After examining the code for
# the CartPole environment, I found that the threshold on the angle of the pole
# before the environment considers itself done is 12 degrees. Obviously, with a
# random selection policy, this will be hit very quickly.
for i_episode in range(20):
  env.reset()
  for t in range(100):
      env.render()
      action = env.action_space.sample() # Choose randomly between push left or
                                         # push right
      observation, reward, done, info = env.step(action)
      # Just take a look a the first observation:
      if (i_episode == 0) and (t == 0):
        print("Initial State:")
        print("\tCart Position: {}\n\tCart Velocity: {}".format( \
                  observation[0], observation[1]))
        print("\tPole Angle: {}\n\tPole Velocity: {}\n".format( \
                  observation[2], observation[3]))

      if done:
        if (i_episode == 0):
          print("Episode finished after {} timesteps\n".format(t+1))
          print("Final State:")
          print("\tCart Position: {}\n\tCart Velocity: {}".format( \
                  observation[0], observation[1]))
          print("\tPole Angle: {}\n\tPole Velocity: {}\n".format( \
                  observation[2], observation[3]))
        break;
env.close()
show_video()

Initial State:
	Cart Position: -0.024594046115470173
	Cart Velocity: 0.19481128457963176
	Pole Angle: 0.010110491143967806
	Pole Velocity: -0.23986941257371275

Episode finished after 13 timesteps

Final State:
	Cart Position: 0.1474584708655252
	Cart Velocity: 0.9852380681873425
	Pole Angle: -0.24563535846873222
	Pole Velocity: -1.6736593526331998



In [52]:
# Define the TensorFlow model we'll use to learn how to keep the pole up.
def forward_pass(x):
  y = tf.layers.dense(x, 4, activation='relu', kernel_initializer=tf.random_normal_initializer(), bias_initializer=tf.zeros_initializer())
  y = tf.layers.dense(y, 4, activation='relu', kernel_initializer=tf.random_normal_initializer(), bias_initializer=tf.zeros_initializer())
  y = tf.layers.dense(y, 4, activation='relu', kernel_initializer=tf.random_normal_initializer(), bias_initializer=tf.zeros_initializer())
  y = tf.layers.dense(y, 2, kernel_initializer=tf.random_normal_initializer(), bias_initializer=tf.zeros_initializer())
  probs = tf.nn.softmax(y)
  return probs

inputs = np.random.rand(10, 4)
tf.reset_default_graph()
with tf.Session() as sess:
  x = tf.placeholder(tf.float32, shape=(10, 4))
  probs = forward_pass(x)

  sess.run(tf.initialize_all_variables())

  probs = sess.run(probs, feed_dict={x: inputs})
  print("Outputs: {}".format(probs))

Outputs: [[9.7633255e-01 2.3667514e-02]
 [9.9989104e-01 1.0899893e-04]
 [9.5241082e-01 4.7589116e-02]
 [9.9525082e-01 4.7491649e-03]
 [9.8786724e-01 1.2132736e-02]
 [8.7377512e-01 1.2622488e-01]
 [8.6599147e-01 1.3400859e-01]
 [9.9997604e-01 2.3907542e-05]
 [9.3069655e-01 6.9303460e-02]
 [9.9884605e-01 1.1539449e-03]]
